# Information Retrieval and Web Analytics: Evaluation

## Rank-Based Metrics


Welcome to the second practical session! In this session you are going to implement some common metrics used for the evaluation of an information retrieval system.


### Unranked retrieval evaluation: Precision and Recall

**Precision:** Precision is the fraction of the retrieved documents (the set A) which is relevant.

$$\begin{equation}
  Precision=\frac{{|R \cap A|}}{{|A|}}
\end{equation} 
$$



**Recall:** is the fraction of the relevant documents (the set R) which has been retrieved.

$$\begin{equation}
  Recall=\frac{{|R \cap A|}}{{|R|}}
\end{equation} 
$$

<img src="images/prec_rec.png" style="width:450px;height:300px;">
<caption><center> <u> <font color=''> Figure 2 </u><font color=''> : Precision and Recall</caption> 

The above definitions of precision and recall assumes that all docs in the set A have been examined. 
However, the user is not usually presented with all docs in the answer set A at once. User sees a ranked set of documents and examines them starting from the top. Thus, precision and recall vary as the user proceeds with their examination of the set A.


If we want to benchmark different systems we need:

- A collection of documents that have to be representative of the collection we expect to be in reality;
- A collection of information needs (have to be representative of the collection we expect to be in reality);
- Human relevance assessment for (information need, document) pairs.


Information retrieval systems (like search engine) can express the relevance for a (information need, document pair) in a binary way or through multiple levels.

Here follow some common metrics used for the evaluation of a retrieval system:

### Rank-Based Measures
- ##### Binary relevance
    - Precision@K (P@K)
    - Average Precision@K (P@K)
    - Mean Average Precision (MAP) 
    - Mean Reciprocal Rank (MRR)
    
- ##### Multiple levels of relevance
    - Normalized Discounted Cumulative Gain (NDCG)

Notice that we only mentioned precision but not recall. Indeed, by returning all the documents for a query will result in a trivial 100% recall, Thus recall by itself is commonly not used as a metric in this context.
    
We are going to test the above metrics on a ranking of results which is stored in the ```inputs/test_predictions.csv``` file. The prediction dataset contains:

- q_id: query id.
- doc_id: document id.
- predicted_relevance: relevance predicted through a ranking algorithm.
- y_true: actual score of the document for the query (ground truth).

### 1 - Packages 

In [1]:
import pandas as pd
import numpy as np

### 2 - Load data into memory

In [2]:
search_results = pd.read_csv("inputs/test_predictions.csv")
search_results.head()

,q_id,doc_id,predicted_relevance,y_true
0,0,0,-0.637926,2.0
1,0,1,-0.824241,1.0
2,0,2,-1.358856,3.0
3,0,3,-0.096755,1.0
4,0,4,-1.268338,0.0


Notice that out ground truth consists of multiple levels:

In [3]:
print("The ground truth of our dataset is composed of {} Relevance Levels: {}" .format(len(search_results["y_true"].unique()), sorted(search_results["y_true"].unique())))

The ground truth of our dataset is composed of 5 Relevance Levels: [0.0, 1.0, 2.0, 3.0, 4.0]


To compute *Precision@K, Mean Average Precision and Mean Reciprocal Rank*, we need binary relevance (relevant-1, not relevant-0).


To simplify the task, we will consider as **relevant all documents that have actual score (y_true) equal or higher than $2$, and not-relevant the remaining documents**. 

Add a column ```bin_y_true``` following the above rule to ```search_results```.

In [4]:
search_results["bin_y_true"] = search_results["y_true"].apply(lambda y:1 if y>2 else 0)
search_results.head()

,q_id,doc_id,predicted_relevance,y_true,bin_y_true
0,0,0,-0.637926,2.0,0
1,0,1,-0.824241,1.0,0
2,0,2,-1.358856,3.0,1
3,0,3,-0.096755,1.0,0
4,0,4,-1.268338,0.0,0


### 3 - Metrics


#### Precision@K (P@K)

Precision at k **(P@k) measures the number of relevant results among the top k documents**. It assess whether the users are getting relevant documents at the top of the ranking or not.

A drawback of this metric is that it fails to take into account the positions of the relevant documents among the top k.

Implement the function ```precision_at_k(y_true, y_score, k)``` that takes as input the true relevance labels, the predicted score, the number of docs to consider k and compute the precision as $k$.

Steps:
1. use ```np.argsort``` and [::1] to obtain the list of indexes of the predicted score sorted in descending order.
2. use the indexes of point 1. to sort the actual relevance label of the documents (hint: ```np.take```).
3. consider the top k relevance label of the documents (after the sorting) and retrieve the number of relevant documents (among the top k, i.e., normalise the number of relevant documents by k).

Notice that the Precision@K is computed for a single query and the respective set of retrieved documents.

In [7]:
def precision_at_k(y_true, y_score, k=10):
    '''    
    Parameters
    ----------
    y_true: Ground truth (true relevance labels).
    y_score: Predicted scores.
    k : number of doc to consider.
    
    Returns
    -------
    precision @k : float
    
    '''    
    print(y_score)
    order = np.argsort(y_score)[::1]
    y_true = np.take(y_true,order[:k])
    relevant = sum(y_true==1)
    return relevant/k

In [6]:
y_score

NameError: name 'y_score' is not defined

##### Compute precisio@10 for query with q_id=0

In [8]:
# Check for query 0

current_query = 0
current_query_res = search_results[search_results["q_id"] == current_query] 
k=5

print("==> Precision@{}: {}\n".format(k,
                                precision_at_k(current_query_res["bin_y_true"], current_query_res["predicted_relevance"], k)))


print("\nCheck on the dataset sorted by score:\n")
current_query_res.sort_values("predicted_relevance", ascending=False).head(k)

0     -0.637926
1     -0.824241
2     -1.358856
3     -0.096755
4     -1.268338
         ...   
133    0.460846
134   -0.415846
135    0.374341
136   -1.290106
137    0.205322
Name: predicted_relevance, Length: 138, dtype: float64
==> Precision@5: 0.2


Check on the dataset sorted by score:



,q_id,doc_id,predicted_relevance,y_true,bin_y_true
88,0,88,1.705258,2.0,0
114,0,114,1.116369,2.0,0
63,0,63,1.096797,1.0,0
34,0,34,1.084367,1.0,0
86,0,86,1.082985,3.0,1


In [36]:
k=3
print("==> Precision@{}: {}\n".format(k,
                                precision_at_k(current_query_res["bin_y_true"], current_query_res["predicted_relevance"], k)))


k=10
print("==> Precision@{}: {}\n".format(k,
                                precision_at_k(current_query_res["bin_y_true"], current_query_res["predicted_relevance"], k)))



==> Precision@3: 0.0

==> Precision@10: 0.2



#### Average Precision@k - AP@K


With respect to $P@K$, $AP@K$ gives a better intuition of the model ability of sorting the results for a specific query. It tells how much the relevant documents are concentrated in the highest ranked predictions.

The Average Precision approximates the area under the uninterpolated precision-recall curve.

$$AP@K=\frac{1}{GTP}\sum_k^n{P@K \times rel@K}\tag{1}$$

where: 
- GTP is the total number of ground truth positives;
- P@k is the precision@k 
- rel@k is a relevance function; it retrievs 1 if the document at rank k is relevant and 0 otherwise.

<img src="images/apk.png" style="width:600px;height:250px;">
<caption><center> <u> <font color=''> Figure 1 </u><font color=''>  : Computation of AP <br> (Picture taken from <i>https://towardsdatascience.com/breaking-down-mean-average-precision-map-ae462f623a52</i>)</center></caption>  

Implement the function ```avg_precision_at_k(y_true, y_score, k)``` that takes as input the true relevance labels, the predicted score, the number of docs to consider k and compute the average precision at 𝑘.

Notice that the Precision@K is computed for a single query and the respective set of retrieved documents.

In [40]:
def avg_precision_at_k(y_true, y_score, k=10):
    
    '''
    Parameters
    ----------
    y_true: Ground truth (true relevance labels).
    y_score: Predicted scores.
    k : number of doc to consider.
    
    Returns
    -------
    average precision @k : float
    '''
    
    gtp = np.sum(y_true == 1)
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])         

    ## if all docs are not relevant
    if gtp==0:
        return 0
    n_relevant_at_i = 0
    prec_at_i = 0
    for i in range(len(y_true)):
        if y_true[i] == 1:
            n_relevant_at_i += 1
            prec_at_i += n_relevant_at_i / (i+1)
    return prec_at_i/gtp

##### Compute precisio@10 for query with q_id=0

In [41]:
avg_precision_at_k(np.array(current_query_res["bin_y_true"]), np.array(current_query_res["predicted_relevance"]), 150)
#avg_precision_at_k(np.array([1,0,0,1,1,0]), np.array([0.9,0.8,0.7,0.6,0.5, 0.4]),6)

0.15432511227002754

In [42]:
# Check with average_precision_score of sklearn

from sklearn.metrics import average_precision_score
k = 150
temp = current_query_res.sort_values("predicted_relevance", ascending=False).head(k)
average_precision_score(np.array(temp["bin_y_true"]), np.array(temp["predicted_relevance"][:k]))

0.15432511227002754

In [43]:
# Check with average_precision_score of sklearn

y_true = np.array([1, 1, 0, 1, 0, 0, 1])
y_scores = np.array([7, 6, 5, 4, 3, 2, 1])
assert(average_precision_score(y_true, y_scores) == avg_precision_at_k(y_true, y_scores,10))

In [44]:
avg_precision_at_k(y_true, y_scores,10)

0.8303571428571428

##### Manual check

In [45]:
avg_precision_at_k(np.array(current_query_res["bin_y_true"]), np.array(current_query_res["predicted_relevance"]), 10) #he canviat , hi havia score al final

0.05625

In [46]:
current_query_res.sort_values("predicted_relevance", ascending=False).head(10)

,q_id,doc_id,predicted_relevance,y_true,bin_y_true
88,0,88,1.705258,2.0,0
114,0,114,1.116369,2.0,0
63,0,63,1.096797,1.0,0
34,0,34,1.084367,1.0,0
86,0,86,1.082985,3.0,1
47,0,47,1.081464,0.0,0
55,0,55,1.075457,2.0,0
76,0,76,1.063326,3.0,1
17,0,17,1.016901,2.0,0
58,0,58,0.906784,1.0,0


In [47]:

(1+(2/2)+(3/5)+(4/7)+(5/8)+(6/9))/np.sum(current_query_res["bin_y_true"])

0.5578869047619048

#### Mean Average Precision (MAP)

The **mAP** is simply the mean of all the queries AP. This metric is not computed for a single query as previous metrics but it takes into account all queries.

Above we mentioned that the average precision approximates the area under the uninterpolated precision-recall curve for a single query. As a consequence, the MAP is roughly the average area under the precision-recall curve for a set of queries.

$$mAP=\frac{1}{N}\sum_{i=1}^n{AP_i}\tag{2}$$

<img src="images/map.png" style="width:600px;height:450px;">
<caption><center> <u> <font color=''> Figure 2 </u><font color=''>  : Computation of mAP </center></caption>  

Implement a function ```map_at_k(search_res, k)``` that takes as input the dataset containing search results (list of actual labels, list of predicted scores, list queries) and k, and compute the Mean Average Precision (MAP).

In [54]:
def map_at_k(search_res, k=10):
    '''
    Parameters
    ----------
    search_res: search results dataset containing:
        q_id: query id.
        doc_id: document id.
        predicted_relevance: relevance predicted through LightGBM.
        y_true: actual score of the document for the query (ground truth).
    
    Returns
    -------
    mean average precision @k : float
    '''
    avp = []
    for q in search_res["q_id"].unique(): #loop over all query id
        curr_data = search_res[search_res["q_id"]==q ]  # select data for current query
        avp.append(avg_precision_at_k(np.array(curr_data["bin_y_true"]), np.array(curr_data["predicted_relevance"]),k)) #append average precision for current query
    return np.sum(avp)/len(avp) # return mean average precision

##### Compute mAP@10 for all queries of the dataset

In [56]:
map_k = map_at_k(search_results,10)
map_k

0.15492345071483846

#### Mean Reciprocal Rank (MRR)

Mean Reciprocal Rank is particularly used when we are interested in the first correct answer.

If we define:

- $R_i$ as the ranking for the query $q_i$;
- $S_correct(R_i)$ as the position of the first correct answer in $R_i$
- $K$ as the threshold for ranking position

The reciprocal rank $$RR(R_i)$$ for query $q_i$ is computed as follows:

$$\begin{equation}
  RR(R_i)==\left\{
  \begin{array}{@{}ll@{}}
    \frac{1}{S_{correct}(R_i)}, & \text{if}\ S_{correct}(R_i) $\leq$ K \\
    0, & \text{otherwise}
  \end{array}\right.
  \tag{3}
\end{equation} 
$$


The Mean Reciprocal Rank (MRR) can be defined as the mean of the RR for all queries:

$$\begin{equation}
  MRR(R_i)==\frac{1}{N}\sum_{i=1}^N{RR(R_i)}
\end{equation} 
\tag{4}
$$

where $N$ is the total number of queries (and rankings since we have a ranking per query).

Implement the function ```rr_at_k(y_true, y_score, k)``` that computes the Reciprocal Rank at the threshold $k$ for a single query and then compute the MRR@K for k=3, 5 and 10.

In [57]:
def rr_at_k(y_true, y_score, k=10):
    '''
    Parameters
    ----------
    y_true: Ground truth (true relevance labels).
    y_score: Predicted scores.
    k : number of doc to consider.
    
    Returns
    -------
    Reciprocal Rank for qurrent query
    '''

    order = np.argsort(y_score)[::-1] # get the list of indexes of the predicted score sorted in descending order.
    y_true =np.take(y_true,order[:k]) # sort the actual relevance label of the documents based on predicted score(hint: np.take) and take first k.
    if np.sum(y_true)== 0: # if there are not relevant doument return 0
        return 0
    return 1/(np.argmax(y_true == 1) +1 ) # hint: to get the position of the first relevant document use "np.argmax"



In [58]:
y_true = np.array([0,1,0,1,1])
score = np.array([0.9, 0.5, 0.6, 0.7, 0.2])
rr_at_k(y_true, score,5)

0.5

##### Make some test with the query with q_id = 8 to check if your function is working properly.


In [59]:
current_query = 8
current_query_res = search_results[search_results["q_id"] == current_query] 
current_query_res.sort_values("predicted_relevance", ascending=False).head(10)

,q_id,doc_id,predicted_relevance,y_true,bin_y_true
1067,8,52,0.115248,0.0,0
1039,8,24,-0.046405,0.0,0
1028,8,13,-0.404693,0.0,0
1051,8,36,-0.493206,1.0,0
1066,8,51,-0.701708,1.0,0
1034,8,19,-0.755329,0.0,0
1015,8,0,-0.802263,2.0,0
1025,8,10,-0.827835,0.0,0
1031,8,16,-0.836900,0.0,0
1022,8,7,-0.878972,0.0,0


In [60]:
labels = np.array(search_results[search_results['q_id'] == 8]["bin_y_true"])
scores = np.array(search_results[search_results['q_id'] == 8]["predicted_relevance"])
np.round(rr_at_k(labels, scores, 10),4)

0

##### Compute the MRR@K for k=3,5,and 10.

In [62]:
mrr = {}
for k in [3,5,10]:
    RRs = []
    for q in search_results['q_id'].unique() : # loop over all query ids
        labels = np.array(search_results[search_results['q_id'] == q]["bin_y_true"]) # get labels for current query
        scores =  np.array(search_results[search_results['q_id'] == q]["predicted_relevance"])# get predicted score for current query
        RRs.append(rr_at_k(labels,scores,k)) # append RR for current query
    mrr[k] = np.round(float(sum(RRs)/len(RRs)),4) # Mean RR at current k

In [63]:
mrr

{3: 0.3012, 5: 0.3121, 10: 0.3213}

### Multiple levels of relevance metrics

#### NDCG - Normalized Discounted Cumulative Gain

**NDCG** also works if document relevances are a real number, i.e., when each document's relevance is note expressed in a binary form (relevant or non-relevant).

This metric is especially used with machine learning based approaches, like Learning To Rank, and it takes values between $0$ (very poor/bad ranking) and $1$ (optimal ranking). 

Before defining $NDCG$, let's talk about **DCG**.
**DCG** is based on the following assumptions:

- Highly relevant documents are more useful when appearing earlier in a search engine result list (have higher ranks)
- Highly relevant documents are more useful than marginally relevant documents, which are in turn more useful than non-relevant documents.

$DCG$ is based on the notion of **CG (Cumulative Gain)**. 

**Cumulative Gain (CG):** does not include the position of a result in the consideration of the usefulness of a result set. It is the sum of the relevance values of all results in a search result list (in the ranking). Suppose you were presented with a set of search results for a query and asked to rank each result:

- 0 => Not relevant 
- 1 => Near relevant 
- 2 => Relevant.

If we sum the values for a page of results we will have a measure of the cumulative gain (CG).

$$CG = \sum_{pos=1}^n Rel_{pos}\tag{5}$$

Where $Rel_{pos}$ is the graded relevance of $pos^{th}$ document.
Cumulative gain, however, does not reward relevant results that appear higher in the result set (CG function is unaffected by changes in the ordering of search results). To achieve the Discounted cumulative gain (DCG) we must discount results that appear lower.

**Discounted Cumulative Gain (DCG):** The premise of DCG is that highly relevant documents appearing lower in a search result list should be penalized as the graded relevance value is reduced logarithmically proportional to the position of the result.

$$DCG = \sum_{pos=1}^n \frac{Rel_{pos}}{\log_2(pos+1)} = Rel_1 + \sum_{pos=2}^n \frac{Rel_{pos}}{\log_2(pos+1)}\tag{6}$$

An alternative formulation of $DCG$ that places stronger emphasis on retrieving relevant documents is the following:

$$DCG = \sum_{pos=1}^n \frac{2^{Rel_{pos}} -1}{\log_2(pos+1)}\tag{7}$$

The latter formula is commonly used in industry including major web search companies. These two formulations of DCG are the same when the relevance values of documents are binary.

**Normalized DCG (NDCG):** If you calculate DCG for different queries you’ll find that some queries are just harder than others and will produce lower DCG scores than easier queries. Normalization solves this problem by scaling the results based off of the best result seen (Ideal DCG or $IDCG$). This is done by sorting all relevant documents in the corpus by their relative relevance, producing the maximum possible DCG through position $n$, also called Ideal DCG (IDCG) through that position (*usually it is the ground truth*). 

$$NDCG_{pos} = \frac{DCG_{pos}}{iDCG}\tag{8}$$

<img src="images/ndcg.png" style="width:650px;height:450px;">
<caption><center> <u> <font color=''> Figure 2 </u><font color=''> : Computation of NDCG </br> 
    (Picture taken from https://medium.com/swlh/rank-aware-recsys-evaluation-metrics-5191bba16832)</center></caption>  

Implement the functions: 
- ```dcg_at_k(y_score, y_true, k)``` based on formula $7$  
- ```ndcg_at_k(y_score, y_true, k)```

Compute:
- the $NDCG@10$ for query with ```q_id=0```
- the average $NDCG@10$ (considering all queries/rankings).

In [68]:
def dcg_at_k(y_true, y_score,  k=10):
    order = np.argsort(y_score)[::-1] # get the list of indexes of the predicted score sorted in descending order.
    y_true = np.take(y_true, order[:k]) # sort the actual relevance label of the documents based on predicted score(hint: np.take) and take first k.
    gain = 2**y_true -1 # Compute gain (use formula 7 above)
    discounts = np.log2(np.arange(len(y_true))+2) # Compute denominator
    return np.sum(gain / discounts) #return dcg@k


def ndcg_at_k(y_true, y_score, k=10):    
    dcg_max = dcg_at_k(y_true, y_true, k) # Ideal dcg
    if not dcg_max:
        return 0
    return np.round(dcg_at_k(y_true, y_score, k)/dcg_max,4)  # return ndcg@k

##### the  𝑁𝐷𝐶𝐺@10 for query with q_id=0

In [69]:
q_id = 0
k = 10
labels = np.array(search_results[search_results['q_id'] == q_id]["y_true"])
scores = np.array(search_results[search_results['q_id'] == q_id]["predicted_relevance"])
ndcg_k = np.round(ndcg_at_k(labels, scores, k),4)
print("ndcg@{} for query with q_id={}: {}".format(k,q_id,ndcg_k))




ndcg@10 for query with q_id=0: 0.4392


##### the average  𝑁𝐷𝐶𝐺@10  (considering all queries/rankings).

In [72]:
ndcgs = []
k=10
for q in search_results['q_id'].unique(): # loop over all query ids
    labels = np.array(search_results[search_results['q_id'] == q]["y_true"]) ## get labels for current query
    scores = np.array(search_results[search_results['q_id'] == q]["predicted_relevance"]) # get predicted score for current query
    ndcgs.append(ndcg_at_k(labels,scores,k)) # append NDCG for current query

avg_ndcg = np.round(float(sum(ndcgs)/len(ndcgs)),4) # Compute average NDCG
print("Average ndcg@{}: {}".format(k,avg_ndcg))



Average ndcg@10: 0.4646
